In [8]:
import torch
import torch.nn as nn
from torch.autograd import Variable as V
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import time
from models import *
from helpers import *
import matplotlib.pyplot as plt

# New stuff.
from torch.distributions import Normal
from torch.distributions.kl import kl_divergence
DO_VAE=True
BATCH_SIZE = 100

In [2]:
# setup
train_dataset = datasets.MNIST(root='./data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)
test_dataset = datasets.MNIST(root='./data/',
                           train=False, 
                           transform=transforms.ToTensor())
torch.manual_seed(3435)
if DO_VAE:
    train_img = torch.stack([torch.bernoulli(d[0]) for d in train_dataset])
    test_img = torch.stack([torch.bernoulli(d[0]) for d in test_dataset])
else:
    train_img = torch.stack([d[0] for d in train_dataset])
    test_img = torch.stack([d[1] for d in test_dataset])
train_img = torch.squeeze(train_img)
test_img = torch.squeeze(test_img)
train_label = torch.LongTensor([d[1] for d in train_dataset])
test_label = torch.LongTensor([d[1] for d in test_dataset])

val_img = train_img[-10000:].clone()
val_label = train_label[-10000:].clone()
train_img = train_img[:-10000]
train_label = train_label[:-10000]

train = torch.utils.data.TensorDataset(train_img, train_label)
val = torch.utils.data.TensorDataset(val_img, val_label)
test = torch.utils.data.TensorDataset(test_img, test_label)
train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE, shuffle=True)

In [3]:
encoder_mlp = MLPEncoder(hidden_dim=200, latent_dim=10)
decoder_mlp = MLPDecoder(hidden_dim=200, latent_dim=10)
vae = NormalVAE(encoder_mlp, decoder_mlp)
model_list = [encoder_mlp, decoder_mlp, vae]
lm_evaluator = LatentModelEvaluator(model_list, batch_sz=BATCH_SIZE, mode='vae')
lm_trainer = LatentModelTrainer(model_list, lrn_rate=0.005, optimizer = optim.SGD,
                                batch_sz=BATCH_SIZE, mode='vae')
lm_trainer.train(train_loader, le=lm_evaluator, val_loader=val_loader,
                num_epochs=10)

CUDA is unavailable...
100 10
CUDA is unavailable...
100 10
Epoch 0, loss: 225.738550, KL: 7.342832, lrn_rate: 0.005000
Validation set: loss: 19806.042852, KL: 558.977668
Epoch 1, loss: 195.058810, KL: 6.045110, lrn_rate: 0.005000
Validation set: loss: 18860.284219, KL: 614.327557
Epoch 2, loss: 186.226257, KL: 7.040453, lrn_rate: 0.005000
Validation set: loss: 17818.897695, KL: 809.571504
Epoch 3, loss: 168.978265, KL: 9.704606, lrn_rate: 0.005000
Validation set: loss: 15494.017871, KL: 1160.747133
Epoch 4, loss: 151.297802, KL: 11.682958, lrn_rate: 0.005000
Validation set: loss: 14452.116279, KL: 1212.032679
Epoch 5, loss: 142.455038, KL: 12.439976, lrn_rate: 0.005000
Validation set: loss: 13940.191318, KL: 1204.167098
Epoch 6, loss: 135.523660, KL: 13.142223, lrn_rate: 0.005000
Validation set: loss: 13099.704658, KL: 1328.734067
Epoch 7, loss: 129.001058, KL: 14.080128, lrn_rate: 0.005000
Validation set: loss: 12566.649082, KL: 1379.403317
Epoch 8, loss: 123.765371, KL: 14.647790, l

In [7]:
print(len(list(encoder_mlp.parameters())))
print(len(list(decoder_mlp.parameters())))
print(len(list(vae.parameters())))

6
4
10


In [20]:
print(len(test_loader))

100


In [4]:
def f(a=1, b=2):
    print(a, b)
d = {'a' : 32}
f(**d, b=2)

32 2


In [19]:
for i,batch in enumerate(train_loader):
    img = batch[0].data.numpy()
    label = batch[1].data.numpy()
    plt.clf()
    plt.imshow(1 - img[0], cmap='gray')
    print(label[0])
    plt.savefig('mnist-ex.png')
    # print(i)
    # print(batch[1])
    break

7
